In [1]:
import json
import numpy as np
import pandas as pd
from clean_ingredients import clean_ingredients
from remove_adj import remove_adj
from scipy import sparse
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
get_ipython().magic(u'matplotlib inline')  
data = json.load(open('/Users/guozhiqi-seven/Downloads/train.json')) 

In [2]:
def modified_ingredients_per_cuisine(data):
    IngrePerCuisineDict = {}
    cuisines = []
    all_ingredients = []
    
    for i in range(len(data)):       
        thiscuisine = data[i]['cuisine']
        
        #pre-clean 
        thisingredients = data[i]['ingredients']
        #thisingredients = remove_adj(thisingredients)
        #thisingredients = clean_ingredients(thisingredients)    
        
        if thiscuisine not in IngrePerCuisineDict.keys():
            cuisines.append(thiscuisine)
            IngrePerCuisineDict[thiscuisine] = thisingredients           
        else: 
            ingrelist = IngrePerCuisineDict[thiscuisine]
            ingrelist.extend(thisingredients)
            IngrePerCuisineDict[thiscuisine] = ingrelist
                 
        all_ingredients.extend(thisingredients)
        
    all_ingredients = list(set(all_ingredients)) # unique list of ALL ingredients
    num_unique = len(all_ingredients)
    num_cuisines = len(cuisines)
    
    return IngrePerCuisineDict,cuisines,num_unique,num_cuisines,all_ingredients

In [3]:
IngrePerCuisineDict,cuisines,num_unique,num_cuisines,all_ingredients=modified_ingredients_per_cuisine(data) 

In [4]:
all_ingredients[:5]

['beef base', 'accent', 'ground peppercorn', 'chicharron', 'squid tube']

In [5]:
def modified_count_matrix_recipe(data,all_ingredients): 
    tfm = np.zeros((len(data),len(all_ingredients)))    
    for ii in range(len(data)):       
        thisrecipe = data[ii]['ingredients']
        thisrecipe = remove_adj(thisrecipe)
        thisrecipe = clean_ingredients(thisrecipe)
        for ingredient in thisrecipe:
            if ingredient in all_ingredients:
                jj = all_ingredients.index(ingredient) 
                tfm[ii,jj] += 1
    return tfm

In [6]:
#Clean&Stemming all ingredients
#Remove unnecessary numbers&symbols&blank space
#Causion: One should using remove_adj first then clean_ingredients
#Reduced ingredients to 4538

all_ingredients = remove_adj(all_ingredients)
all_ingredients = clean_ingredients(all_ingredients)


In [7]:
recipe_tfm=modified_count_matrix_recipe(data,all_ingredients)   

In [8]:
def tfm_tfidf(tfm):
    countsMatrix = sparse.csr_matrix(tfm)
    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(tfm)  
    return tfidf 

In [9]:
data = json.load(open('/Users/guozhiqi-seven/Downloads/train.json')) 
train_cuisine = [recipe['cuisine'] for recipe in data]
recipe_length = [len(recipe['ingredients']) for recipe in data]
labelencoder = LabelEncoder()
cuisine_label = labelencoder.fit_transform(train_cuisine)

In [10]:
tfm_recipe_tfidf = tfm_tfidf(recipe_tfm)

In [11]:
# Adding recipe length to the matrix
recipe_length = np.array(recipe_length)
X_data = np.hstack((tfm_recipe_tfidf.toarray(),recipe_length.reshape(len(recipe_length),1)))

## 暂时不跑 ++++++++++++++++++++++++

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
test_size = [1/30*x for x in list(range(1,30))]
LOG = []
tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_data)

for test in test_size:
    X_train, X_test, y_train, y_test = train_test_split(X_data,cuisine_label,\
                                                    test_size=test,random_state=0)
    labelencoder = LabelEncoder()
    y_train = labelencoder.fit_transform(y_train)
    y_test = labelencoder.transform(y_test)    
    LogReg = LogisticRegression(C=10)
    LogReg = LogReg.fit(X_train,y_train) 
    print('test_size = ',test, 'Score:',LogReg.score(X_test, y_test) )
    LOG.append(LogReg.score(X_test, y_test))

test_size =  0.03333333333333333 Score: 0.781297134238
test_size =  0.06666666666666667 Score: 0.78431372549
test_size =  0.1 Score: 0.782302664656
test_size =  0.13333333333333333 Score: 0.779411764706
test_size =  0.16666666666666666 Score: 0.781867551667
test_size =  0.2 Score: 0.782023884349
test_size =  0.23333333333333334 Score: 0.781165822648
test_size =  0.26666666666666666 Score: 0.776562647308
test_size =  0.3 Score: 0.77432330512
test_size =  0.3333333333333333 Score: 0.776587720622
test_size =  0.36666666666666664 Score: 0.772970378497
test_size =  0.4 Score: 0.772030169705
test_size =  0.43333333333333335 Score: 0.769087955442
test_size =  0.4666666666666667 Score: 0.768990410516
test_size =  0.5 Score: 0.766430331372
test_size =  0.5333333333333333 Score: 0.765191156366
test_size =  0.5666666666666667 Score: 0.762633657216
test_size =  0.6 Score: 0.761282212445
test_size =  0.6333333333333333 Score: 0.760112738677
test_size =  0.6666666666666666 Score: 0.75512897873
test_

In [44]:
thefile = open('Feature_Engineering_LR.txt', 'w')
for item in LOG:
    thefile.write("%s\n" % item)

## 空隙++++++++++++++++++++++++

In [21]:
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    cuisine_label,test_size=0.25,random_state=0,stratify=cuisine_label)
'''

In [22]:
'''
from sklearn.linear_model import LogisticRegression
C_penalty = [10**i for i in range(-3,3)]
for C_para in C_penalty:
    LogReg = LogisticRegression(C=C_para)
    LogReg = LogReg.fit(X_train,y_train) 
    print('C = ',C_para, 'Score:',LogReg.score(X_test, y_test) ) 
'''

C =  0.001 Score: 0.286705551086
C =  0.01 Score: 0.455048270314
C =  0.1 Score: 0.654967819791
C =  1 Score: 0.748793242156
C =  10 Score: 0.766894609815
C =  100 Score: 0.753419147224


In [12]:
import xgboost as xgb 
#from xgboost import XGBClassifier  
from sklearn.metrics import accuracy_score

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    cuisine_label,test_size=0.25,random_state=0,stratify=cuisine_label)

In [14]:
xgb_train = xgb.DMatrix(X_train, label=y_train)
xgb_test = xgb.DMatrix(X_test, label=y_test) 

In [31]:
'''
param = {}
#use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.008
param['max_depth'] = 15
param['silent'] = 1
param['min_child_weight'] = 8
param["subsample"] = 0.7
param["colsample_bytree"] = 0.7
param['nthread'] = 1
param['num_class'] = 20

watchlist = [ (xgb_train,'train'), (xgb_test, 'test') ]
num_round = 2
bst = xgb.train(param, xgb_train, num_round, watchlist, early_stopping_rounds=80);
# get prediction
pred = bst.predict(xgb_test);
print ('predicting, classification error=%f' % (sum( int(pred[i]) != y_test[i] for i in range(len(y_test))) / float(len(y_test))))
'''

[0]	train-merror:0.454408	test-merror:0.478781
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 80 rounds.
[1]	train-merror:0.412169	test-merror:0.438656
predicting, classification error=0.438656


In [16]:
param = {}

param['bst:max_depth'] = 20
param['bst:eta'] = 0.2
param['silent'] = 0
param['objective'] = 'multi:softmax'
param['num_class'] = 20
param['nthread'] = 2
param['eval_metric'] = 'merror'

watchlist = [(xgb_train,'train'), (xgb_test, 'test')]

num_round = 200
bst = xgb.train(param, xgb_train, num_round, watchlist)


pred = bst.predict(xgb_test)
print ('predicting, classification error=%f' % (sum( int(pred[i]) != y_test[i] for i in range(len(y_test))) / float(len(y_test))))


[0]	train-merror:0.418706	test-merror:0.447908
[1]	train-merror:0.351257	test-merror:0.392498
[2]	train-merror:0.32464	test-merror:0.370977
[3]	train-merror:0.315018	test-merror:0.364642
[4]	train-merror:0.302179	test-merror:0.357804
[5]	train-merror:0.293765	test-merror:0.349356
[6]	train-merror:0.283305	test-merror:0.343624
[7]	train-merror:0.276064	test-merror:0.340306
[8]	train-merror:0.26822	test-merror:0.337188
[9]	train-merror:0.261046	test-merror:0.334875
[10]	train-merror:0.25538	test-merror:0.332462
[11]	train-merror:0.248575	test-merror:0.327534
[12]	train-merror:0.24462	test-merror:0.325121
[13]	train-merror:0.239859	test-merror:0.321098
[14]	train-merror:0.234462	test-merror:0.318685
[15]	train-merror:0.229098	test-merror:0.314863
[16]	train-merror:0.224472	test-merror:0.313355
[17]	train-merror:0.220617	test-merror:0.310036
[18]	train-merror:0.216996	test-merror:0.307824
[19]	train-merror:0.213845	test-merror:0.306114
[20]	train-merror:0.209889	test-merror:0.303399
[21]	t

## 间隔

In [51]:
tfm = modified_count_matrix_recipe(data,all_ingredients)

In [ ]:
import sklearn.feature_selection
mutual_info = sklearn.feature_selection.mutual_info_classif(tfm,cuisine_label)
mu = mutual_info.argsort()
mutual_information = []
for i in range(len(mu)):
    mutual_information.append([mutual_info[mu[i]],mu[i]])
top_mutual = (mutual_info.shape - sum(mutual_info == 0))
index = [x[1] for x in mutual_information[-top_mutual:]]
ingred_mutual = [list(ingredients)[ind] for ind in index]